In [ ]:
import os, json
import numpy as np
from tqdm import tqdm
from collections import Counter
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler, Dataset
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from facenet_pytorch import MTCNN
import mediapipe as mp

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.backends.cudnn.benchmark = True
print(f"Device: {device}")

# ---------------------------
# Paths & Dataset
# ---------------------------
train_path = r"C:\Ashvin\AI ML\Project\Emotion Detection\Data\train"
test_path  = r"C:\Ashvin\AI ML\Project\Emotion Detection\Data\test"

train_raw = datasets.ImageFolder(train_path)
test_raw  = datasets.ImageFolder(test_path)

with open("classes.json", "w") as f:
    json.dump(train_raw.classes, f)

# ---------------------------
# Face detection & landmarks
# ---------------------------
mtcnn = MTCNN(image_size=128, margin=10, keep_all=False, device=device)
mp_face = mp.solutions.face_mesh

def preprocess_faces(dataset, split):
    print(f" Preprocessing {split} set...")
    faces, labels, landmarks_all = [], [], []
    with mp_face.FaceMesh(static_image_mode=True, max_num_faces=1) as fm:
        for img, label in tqdm(dataset):
            img_rgb = img.convert('RGB')
            face_rgb = mtcnn(img_rgb)
            if isinstance(face_rgb, list):
                face_rgb = face_rgb[0] if len(face_rgb) > 0 else None
            if face_rgb is None:
                continue
            np_face = (face_rgb.permute(1,2,0).cpu().numpy() * 255).astype(np.uint8)
            results = fm.process(np_face)
            if results.multi_face_landmarks:
                landmarks = [(lm.x, lm.y, lm.z) for lm in results.multi_face_landmarks[0].landmark]
                if len(landmarks) == 468:
                    faces.append(face_rgb)
                    labels.append(label)
                    landmarks_all.append(landmarks)
    print(f" Collected {len(faces)} faces for {split}")
    return torch.stack(faces), torch.tensor(labels), landmarks_all

if os.path.exists('ed_train_data.pt') and os.path.exists('ed_test_data.pt'):
    print(" Loading preprocessed data...")
    train_data = torch.load('ed_train_data.pt')
    test_data  = torch.load('ed_test_data.pt')
    train_faces, train_labels, train_landmarks = train_data['faces'], train_data['labels'], train_data['landmarks']
    test_faces, test_labels, test_landmarks    = test_data['faces'],  test_data['labels'],  test_data['landmarks']
else:
    train_faces, train_labels, train_landmarks = preprocess_faces(train_raw, 'train')
    test_faces, test_labels, test_landmarks    = preprocess_faces(test_raw, 'test')
    torch.save({'faces': train_faces, 'labels': train_labels, 'landmarks': train_landmarks}, 'ed_train_data.pt')
    torch.save({'faces': test_faces,  'labels': test_labels,  'landmarks': test_landmarks},  'ed_test_data.pt')

# ---------------------------
# Compute global mean/std for landmarks
# ---------------------------
all_landmarks = np.array([np.array(l).flatten() for l in train_landmarks])
lm_mean = torch.tensor(all_landmarks.mean(axis=0), dtype=torch.float)
lm_std  = torch.tensor(all_landmarks.std(axis=0), dtype=torch.float) + 1e-6

# ---------------------------
# Dataset with stronger augmentation
# ---------------------------
class EmotionDataset(Dataset):
    def __init__(self, faces, labels, landmarks, augment=True, lm_mean=None, lm_std=None):
        self.faces = faces
        self.labels = labels
        self.landmarks = landmarks
        self.augment = augment
        self.lm_mean = lm_mean
        self.lm_std = lm_std
        self.aug_transform = transforms.Compose([
            transforms.RandomResizedCrop(128, scale=(0.8,1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(0.2,0.15,0.15),
            transforms.RandomApply([transforms.RandomAffine(10, translate=(0.1,0.1))], p=0.3),
            transforms.RandomErasing(p=0.2, scale=(0.02,0.1)),  # new
            transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
        ])
        self.base_transform = transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])

    def __len__(self): return len(self.faces)

    def __getitem__(self, idx):
        face = self.faces[idx]
        landmarks = torch.tensor(np.array(self.landmarks[idx]).flatten(), dtype=torch.float)
        landmarks = (landmarks - self.lm_mean) / self.lm_std
        face = self.aug_transform(face) if self.augment else self.base_transform(face)
        return face, landmarks, self.labels[idx]

train_dataset = EmotionDataset(train_faces, train_labels, train_landmarks, augment=True, lm_mean=lm_mean, lm_std=lm_std)
test_dataset  = EmotionDataset(test_faces,  test_labels,  test_landmarks, augment=False, lm_mean=lm_mean, lm_std=lm_std)

# ---------------------------
# DataLoader with balancing
# ---------------------------
class_counts = Counter(train_labels.numpy())
median = np.median(list(class_counts.values()))
class_weights = {k: median/v for k,v in class_counts.items()}
sample_weights = [class_weights[l.item()] for l in train_labels]
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=64, sampler=sampler)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# ---------------------------
# Model: deeper classifier & landmark MLP
# ---------------------------
class EmotionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,64,3,padding=1), nn.BatchNorm2d(64), nn.LeakyReLU(0.1), nn.MaxPool2d(2), nn.Dropout(0.3),
            nn.Conv2d(64,128,3,padding=1), nn.BatchNorm2d(128), nn.LeakyReLU(0.1), nn.MaxPool2d(2), nn.Dropout(0.4),
            nn.Conv2d(128,256,3,padding=1), nn.BatchNorm2d(256), nn.LeakyReLU(0.1), nn.AdaptiveAvgPool2d((4,4)), nn.Dropout(0.4)
        )
        self.landmark_mlp = nn.Sequential(
            nn.Linear(1404, 256), nn.BatchNorm1d(256), nn.LeakyReLU(0.1), nn.Dropout(0.4)
        )
        self.classifier = nn.Sequential(
            nn.Linear(256*4*4 + 256, 256), nn.BatchNorm1d(256), nn.LeakyReLU(0.1), nn.Dropout(0.4),
            nn.Linear(256,128), nn.BatchNorm1d(128), nn.LeakyReLU(0.1), nn.Dropout(0.3),
            nn.Linear(128,7)
        )

    def forward(self, x, landmarks):
        x = self.features(x)
        x = torch.flatten(x,1)
        l = self.landmark_mlp(landmarks)
        x = torch.cat([x, l], dim=1)
        return self.classifier(x)

model = EmotionCNN().to(device)

# ---------------------------
# Optimizer & scheduler
# ---------------------------
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)  # reduced smoothing
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

# ---------------------------
# Training loop
# ---------------------------
epochs, patience, best_f1, counter = 50, 10, 0, 0
print("\n Starting training")

for epoch in range(epochs):
    model.train()
    running_loss=0.0
    for images, landmarks, labels in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]"):
        images, landmarks, labels = images.to(device), landmarks.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images, landmarks)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss/len(train_loader)

    # Eval with TTA
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, landmarks, labels in test_loader:
            images, landmarks = images.to(device), landmarks.to(device)
            outputs = model(images, landmarks)
            outputs_flip = model(torch.flip(images,dims=[3]), landmarks)
            outputs_mean = (outputs + outputs_flip)/2
            _, preds = outputs_mean.max(1)
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.numpy())

    val_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    val_acc= accuracy_score(y_true, y_pred)
    print(f"Epoch [{epoch+1}] Loss:{avg_loss:.4f} | Val Acc:{val_acc:.4f} | Val F1:{val_f1:.4f}")

    scheduler.step()

    if val_f1>best_f1:
        best_f1, counter = val_f1, 0
        torch.save(model.state_dict(), "ed_final_best_model.pth")
        print(" Saved best model")
    else:
        counter+=1
        if counter>=patience:
            print(" Early stopping")
            break

# ---------------------------
# Final evaluation
# ---------------------------
model.load_state_dict(torch.load("ed_final_best_model.pth"))
model.eval()

def evaluate(loader):
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, landmarks, labels in loader:
            images, landmarks = images.to(device), landmarks.to(device)
            outputs = model(images, landmarks)
            outputs_flip = model(torch.flip(images,dims=[3]), landmarks)
            outputs_mean = (outputs + outputs_flip)/2
            _, preds = outputs_mean.max(1)
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.numpy())
    return {
        'accuracy': accuracy_score(y_true,y_pred),
        'precision': precision_score(y_true,y_pred,average='macro', zero_division=0),
        'recall': recall_score(y_true,y_pred,average='macro', zero_division=0),
        'f1': f1_score(y_true,y_pred,average='macro', zero_division=0)
    }

train_metrics = evaluate(train_loader)
test_metrics  = evaluate(test_loader)

print("\n Final Performance:")
print(f"Train Accuracy: {train_metrics['accuracy']:.4f}")
print(f"Test Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"Test F1-Score:  {test_metrics['f1']:.4f}")

with open("ed_final_train_metrics.json","w") as f: json.dump(train_metrics,f,indent=2)
with open("ed_final_test_metrics.json","w") as f: json.dump(test_metrics,f,indent=2)

print("  Done!") 

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cpu
 Loading preprocessed data...

 Starting training


Epoch [1/50]: 100%|██████████| 191/191 [09:31<00:00,  2.99s/it]


Epoch [1] Loss:1.8585 | Val Acc:0.3083 | Val F1:0.2495
 Saved best model


Epoch [2/50]: 100%|██████████| 191/191 [12:22<00:00,  3.89s/it]


...

Epoch [41] Loss:1.2224 | Val Acc:0.5473 | Val F1:0.5069


Epoch [42/50]: 100%|██████████| 191/191 [07:33<00:00,  2.37s/it]


Epoch [42] Loss:1.2075 | Val Acc:0.5364 | Val F1:0.5019


Epoch [43/50]: 100%|██████████| 191/191 [08:30<00:00,  2.67s/it]


Epoch [43] Loss:1.2179 | Val Acc:0.5447 | Val F1:0.5083
 Early stopping

 Final Performance:
Train Accuracy: 0.6172
Test Accuracy:  0.5523
Test F1-Score:  0.5109
  Done!
